In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
import os
# from tutorial:
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

In [ ]:
#'/content/drive/MyDrive/projeto-dl/Dataset_BUSI_with_GT/benign/'
#'/content/drive/MyDrive/projeto-dl/Dataset_BUSI_with_GT/malignant'
#'/content/drive/MyDrive/projeto-dl/Dataset_BUSI_with_GT/normal'

Versão tutorial:

In [ ]:
# From tutorial:
model = torchvision.models.mobilenet_v2(pretrained=True)
model.eval()
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_life_interpreter("model.ptl")

Minha adaptação:

In [ ]:
# Para executar com a GPU:
#DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = torchvision.models.mobilenet7(pretrained=False) # uso uma pré-treinada ou não?

In [ ]:
class ImgDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.transform = transforms.Compose([
          # talvez devesse cortar como o livro sugere, mas tem segmentações grandes demais pra qualquer tipo de corte, ent vou dxar assim por enquanto
          transforms.Resize([256, 256]), # De acordo com a descrição do dataset, a média de tamanho das imagens é 500x500
          transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image_path']
        mask_path = self.data.iloc[idx]['mask_path']

        image = Image.open(image_path).convert('L')
        mask = Image.open(mask_path).convert('L')

        return self.transform(image), self.transform(mask)